In [ ]:
# Import Libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import geopandas
import numpy as np
import matplotlib.pyplot as plt
#from scipy.stats import spearmanr
#from re import sub
#from decimal import Decimal
#from shapely.geometry import Point, Polygon
import contextily as cx
%config InlineBackend.figure_format ='retina'

In [ ]:
# Import processed judgements_GEOIDS data -- no businesses
jud_geo = pd.read_csv("~/cpal_eviction_public/data/nonzero_jud_geo_no_bus.csv")
jud_geo.head(3)

In [ ]:
# Import cleaned census data -- originally: Selected Housing Characteristics, ACS 1-year -- cleaned by Christine
median_rent = pd.read_csv("~/cpal_eviction_public/data/rentdata_clean.csv")
##median_rent = median_rent.drop(columns=['0500000US48113'])
median_rent.columns = ['GEOID', 'NAME', 'Estimate_Rent']
median_rent = median_rent[median_rent['GEOID']!='Geography']
#median_rent['GEOID'] = median_rent['GEOID'].str.replace('1400000US','')
median_rent['GEOID'] = median_rent['GEOID'].astype(float)
median_rent = median_rent[median_rent['Estimate_Rent']!='-']
median_rent['Estimate_Rent'] = median_rent['Estimate_Rent'].astype(float)
median_rent.head(3)

In [ ]:
# Import census tract shapefile
tx_tract = geopandas.read_file("https://www2.census.gov/geo/tiger/TIGER2020/TRACT/tl_2020_48_tract.zip")
tx_tract = tx_tract[tx_tract['COUNTYFP'].astype(float)==113]
tx_tract = tx_tract.to_crs(epsg = 4326)

In [ ]:
income = pd.read_csv("~/cpal_eviction_public/data/incomedata_clean.csv")


In [ ]:
# Count the number of evictions by census tract
jud_geo_by_tract = pd.DataFrame(np.unique(jud_geo['GEOID_2020'], return_counts=True)).transpose()
jud_geo_by_tract.columns = ['GEOID_2020', 'Num_Evict']

# Find the total and average judgment award by census tract
jud_geo_amt_median = jud_geo[['GEOID_2020','judgment_amt']].groupby(by='GEOID_2020').median()['judgment_amt']
jud_geo_by_tract = pd.merge(jud_geo_by_tract, jud_geo_amt_median, left_on='GEOID_2020', right_on='GEOID_2020', how='outer')
jud_geo_by_tract.columns = ['GEOID_2020', 'Num_Evict', 'Median_Judgment_Amt']
jud_geo_by_tract

In [ ]:
# Convert GEOIDs to float
jud_geo_by_tract['GEOID_2020'] = jud_geo_by_tract['GEOID_2020'].astype(float)
tx_tract['GEOID'] = tx_tract['GEOID'].astype(float)

# Combine with census tracts
tx_tract_jud_geo = pd.merge(tx_tract, jud_geo_by_tract, left_on='GEOID', right_on='GEOID_2020', how='left')
tx_tract_jud_geo.head(3)

In [ ]:
tx_tract_jud_geo_median_rent = pd.merge(tx_tract_jud_geo, median_rent, left_on='GEOID', right_on='GEOID', how='left')
tx_tract_jud_geo_median_rent['Median_Months_Judgment'] = tx_tract_jud_geo_median_rent['Median_Judgment_Amt'] / tx_tract_jud_geo_median_rent['Estimate_Rent']
tx_tract_jud_geo_median_rent.head(1)

In [ ]:
income = income[income['Renter Median Outcome'] > 0]


In [ ]:
tx_tract_jud_geo_median_rent_income = pd.merge(tx_tract_jud_geo_median_rent, income, 
                                               left_on='NAME_y', right_on='Tract', how='left')

In [ ]:
tx_tract_jud_geo_median_rent_income['pct_med_rent_income'] = tx_tract_jud_geo_median_rent_income['Estimate_Rent']*12*100/tx_tract_jud_geo_median_rent_income['Renter Median Outcome']


In [ ]:
gdf_wm = tx_tract_jud_geo_median_rent_income.to_crs(epsg=3857)


In [ ]:
gdf_wm.to_csv('/users/akage/cpal_eviction_public/data/tx_tract_jud_geo_median_rent_income.csv')


In [ ]:
def create_map(df, col_name, map_title, title_size, x_size, y_size, legend_orientation, save_fig, file_name):
    
    ax=df.plot(column=col_name, cmap='OrRd', legend=True, alpha=0.6,  figsize=(x_size,y_size), 
               missing_kwds={"color": "lightgrey", "edgecolor": "grey","hatch": "///",}, 
               edgecolor='k', legend_kwds={'orientation': legend_orientation, 'shrink': 0.8,  'pad': 0.01})
    cx.add_basemap(ax,source=cx.providers.OpenStreetMap.Mapnik, zoom=14)
    cx.add_basemap(ax,source=cx.providers.OpenStreetMap.Mapnik, zoom=12)
    leg = ax.get_legend()

    ax.set_axis_off();
    ax.set_title(map_title, fontsize=title_size);

    if save_fig == True:
        plt.savefig(file_name, dpi=1200, bbox_inches="tight", pad_inches = 0)

In [ ]:
# Number of Evictions
create_map(gdf_wm, 'Num_Evict', 'Number of Evictions by Census Tract', 24, 15, 15, 'vertical', 
           True, '../results/num_evict.png')

In [ ]:
# Median Judgment Amount
create_map(gdf_wm, 'Median_Judgment_Amt', 'Median Judgment Amount ($) by Census Tract', 24, 15, 15, 'vertical', 
           True, '../results/med_jud_amt.png')

In [ ]:
#Rent
create_map(gdf_wm, 'Estimate_Rent', 'Median Contract Rent ($) by Census Tract', 24, 15, 15, 'vertical', 
           True, '../results/med_rent.png')

In [ ]:
# Median Renter Income
create_map(gdf_wm, 'Renter Median Outcome', 'Median Renter Income ($) by Census Tract', 24, 15, 15, 'vertical', 
           True, '../results/med_income.png')

In [ ]:
# Median % Income of Rent
create_map(gdf_wm, 'pct_med_rent_income', 'Percent Rent of Income (%) by Census Tract', 24, 15, 15, 'vertical', 
           True, '../results/med_pct_rent_income.png')